# Autoencoder map diifernt dimension
## Mock Map Dataset


In [1]:
import numpy as np

In [2]:
square="................~............@@@\n##....X........~.....e.......@@@\n###..GJ.........~.@@g$$.....@@.@\n..#...........~~..@..$..@@@@@@@@\n.#.....&......~~..@@@$..@@@@@@@@\n...#...........~.@@@@@@@@@@@@@@@\n.....#...........@@@@@@.@@@@@@@@\n...###........$...@.@@@#.@@@@@@@\n....###@..#.......@@@....@@@@...\n...##@@@@.##......@....##@@.....\n.....@@@@@##...@@@@......@@.....\n.@@@@@@@@@#@@@@@@@@.@..##..#....\n.@@@@@@@@@@.@@@@@@@@@..###.x.&..\n..@@.@.@@@..@@@@@@@@@..##.......\n...@.@@@@@@@@....@@@@#..#...#...\n..@@.@@@@@@@@@..........#####...\n.@@@@@@@@@.@@.........##.#.#.#..\n.@@@@@@@@@..&.....c..#########..\n.@@@@@@@@@......a.f.#.#.#.###...\n..@@@K.............h.########...\n...@@..H.........b...##.######..\n..@@@............$$.##..#######.\n....&......#........########.##.\n..B.........#........##.#..##.#.\n......D...#.#...#....#..####.#..\n$$.............#.i...#.....##...\n...........##....#.##..#..#.....\n..$A.......#.##..##...#....@@@..\n...C.I.....##.....##.#.#...@@@..\n..EF.......##....j..###...@@@@@@\n...........##.......##......@@@.\n................#...##.#...@@@@@"
square_my_symbol="a"
square_enemies="upper"

In [3]:
rectangle="................~............@@@\n##....X........~.....e.......@@@\n###..GJ.........~.@@g$$.....@@.@\n..#...........~~..@..$..@@@@@@@@\n.#.....&......~~..@@@$..@@@@@@@@\n...#...........~.@@@@@@@@@@@@@@@\n.....#...........@@@@@@.@@@@@@@@\n...###........$...@.@@@#.@@@@@@@\n....###@..#.......@@@....@@@@...\n...##@@@@.##......@....##@@.....\n.....@@@@@##...@@@@......@@.....\n.@@@@@@@@@#@@@@@@@@.@..##..#....\n.@@@@@@@@@@.@@@@@@@@@..###.x.&..\n..@@.@.@@@..@@@@@@@@@..##.......\n...@.@@@@@@@@....@@@@#..#...#...\n..@@.@@@@@@@@@..........#####...\n.@@@@@@@@@.@@.........##.#.#.#..\n.@@@@@@@@@..&.....c..#########..\n.@@@@@@@@@......a.f.#.#.#.###...\n..@@@K.............h.########...\n...@@..H.........b...##.######..\n..@@@............$$.##..#######.\n....&......#........########.##.\n..B.........#........##.#..##.#.\n......D...#.#...#....#..####.#..\n$$.............#.i...#.....##...\n...........##....#.##..#..#.....\n..$A.......#.##..##...#....@@@..\n...C.I.....##.....##.#.#...@@@..\n..EF.......##....j..###...@@@@@@\n...........##.......##......@@@.\n................#...##.#...@@@@@"
rectangle_my_symbol="a"
rectangle_enemies="upper"

In [4]:
dataset_symbol=[(square,square_my_symbol,square_enemies),(rectangle,rectangle_my_symbol, rectangle_enemies),(square,square_my_symbol,square_enemies)]

In [5]:
dataset_raw=[]
for elem in dataset_symbol:
    mappa_processed=[]
    for line in elem[0].splitlines():
        line_content=[]
        for symbol in line:
            line_content.append(symbol)
        mappa_processed.append(line_content)

    dataset_raw.append((np.array(mappa_processed),elem[1],elem[2]))


In [6]:
dataset_raw[0]

(array([['.', '.', '.', ..., '@', '@', '@'],
        ['#', '#', '.', ..., '@', '@', '@'],
        ['#', '#', '#', ..., '@', '.', '@'],
        ...,
        ['.', '.', 'E', ..., '@', '@', '@'],
        ['.', '.', '.', ..., '@', '@', '.'],
        ['.', '.', '.', ..., '@', '@', '@']], dtype='<U1'), 'a', 'upper')

## Process dataset

In [7]:
dataset_processed=[]
for image,current_symbol,current_enemies in dataset_raw:
    this_image=[]
    for row in range(len(image[0])):
        this_row=[]
        for column in range(len(image[1])):
            current_cell=image[row][column]

            #Immettere infor per i bonus
            if(current_cell == "#" or current_cell == "@" or current_cell == "!" or current_cell == "&"):
                result = 0.8
            elif(current_cell=="X"):
                if(current_enemies=="upper"):
                    result=0.2
                else:
                    result=0.4
            elif(current_cell=="x"):
                if(current_enemies=="upper"):
                    result=0.4
                else:
                    result=0.2
            elif(current_cell==current_symbol):
                result=0
            elif(current_cell.isupper() and current_enemies=="upper"):
                result = 0.6
            elif(current_cell.islower() and current_enemies=="lower"):
                result = 0.6 
            else:
                result = 0.0
            this_row.append(result)
        this_image.append(this_row)
    dataset_processed.append(np.array(this_image))
dataset_processed=np.array(dataset_processed)

In [8]:
dataset_processed[0]

array([[0. , 0. , 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0.8, ..., 0.8, 0. , 0.8],
       ...,
       [0. , 0. , 0.6, ..., 0.8, 0.8, 0.8],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0. ],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0.8]])

## Autoencoder

In [9]:
#Parameters
latent_dim_encoder=64
max_map_dimension=128
latent_dim_decoder=max_map_dimension*max_map_dimension
# Si producono delle mappe 128x128 anche se sono più piccole poi si tagliano


### Generalize Dataset

In [10]:
dataset_processed_generalized=[]
for elem in dataset_processed:
    extension = np.empty((max_map_dimension,max_map_dimension))
    extension.fill(0.95)
    extension[:elem.shape[0],:elem.shape[1]] = elem
    dataset_processed_generalized.append(extension)
dataset_processed_generalized=np.array(dataset_processed_generalized)

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim_encoder, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(latent_dim_decoder, activation='sigmoid'),
      layers.Reshape((max_map_dimension, max_map_dimension))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [12]:

x_train=dataset_processed_generalized
x_test=dataset_processed_generalized
print(type(x_test))

print (x_train.shape)
print (x_test.shape)

<class 'numpy.ndarray'>
(3, 128, 128)
(3, 128, 128)


In [13]:
autoencoder=Autoencoder()
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(x_train, x_train,
                epochs=1000,
                shuffle=True,
                validation_data=(x_test, x_test))


poch 803/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 804/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 805/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 806/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 807/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 808/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 809/1000
1/1 [==============================] - 0s 126ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 810/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 811/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 812/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0025 - val_los

In [39]:
autoencoder.save_weights('ml_models/autoencoder/autoencoder_weights.wg')
#
# tf.keras.models.save_model(autoencoder,'autoencoder.h5',save_format="tf")

In [14]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [15]:
def post_process_image(decoded_image):
    list_values=[0,0.2,0.4,0.6,0.8,0.95]

    for row in range(len(decoded_image[0])):
        for column in range(len(decoded_image[1])):
            current_cell=decoded_image[row][column]
            differences=[abs(current_cell-elem)for elem in list_values]
            decoded_image[row][column]=list_values[differences.index(min(differences))]
    return decoded_image

In [16]:
x_test[0]

array([[0.  , 0.  , 0.  , ..., 0.95, 0.95, 0.95],
       [0.8 , 0.8 , 0.  , ..., 0.95, 0.95, 0.95],
       [0.8 , 0.8 , 0.8 , ..., 0.95, 0.95, 0.95],
       ...,
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95],
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95],
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95]])

In [17]:
post_process_image(decoded_imgs[0])

array([[0.  , 0.  , 0.  , ..., 0.95, 0.95, 0.95],
       [0.8 , 0.95, 0.  , ..., 0.95, 0.95, 0.95],
       [0.8 , 0.8 , 0.8 , ..., 0.95, 0.95, 0.95],
       ...,
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95],
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95],
       [0.95, 0.95, 0.95, ..., 0.95, 0.95, 0.95]], dtype=float32)

## Mock dataset sequences

In [18]:
# Parametri
max_len_sequence=5
input_output_dim=64
num_lstm_cell=50

In [19]:
sequence=np.array([dataset_processed_generalized[0],dataset_processed_generalized[1],dataset_processed_generalized[2]])
sequence_extra=np.array([dataset_processed_generalized[0],dataset_processed_generalized[1]])

dataset_sequences=np.array([sequence,sequence,sequence,sequence_extra])

x_train_encoded_sequences=[]
y_train_encoded_sequences=[]
for current_sequence in dataset_sequences:
    encoded_imgs = autoencoder.encoder(current_sequence).numpy()
    y_train_encoded_sequences.append(encoded_imgs[-1])
    x_train_encoded_sequences.append(encoded_imgs[:-1])

x_train=np.array(x_train_encoded_sequences)
y_train=np.array(y_train_encoded_sequences)

x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_len_sequence,padding="post"
)


In [20]:
len(x_train)

4

In [21]:
len(x_train[0])

5

## LSTM

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
lstm_model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(max_len_sequence, input_output_dim)),
    #tf.keras.layers.Embedding(input_dim=max_len_sequence, output_dim=input_output_dim, mask_zero=True),
    tf.keras.layers.LSTM(num_lstm_cell),
    tf.keras.layers.Dense(units=input_output_dim)
])

In [29]:
lstm_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

In [30]:
lstm_model.fit(x=x_train, y=y_train,epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 2692.7444 - mse: 2692.7444
Epoch 2/5
1/1 [==============================] - 0s 3ms/step - loss: 2686.6511 - mse: 2686.6511
Epoch 3/5
1/1 [==============================] - 0s 2ms/step - loss: 2679.9180 - mse: 2679.9180
Epoch 4/5
1/1 [==============================] - 0s 3ms/step - loss: 2673.7014 - mse: 2673.7014
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 2666.4680 - mse: 2666.4680


In [40]:
lstm_model.save('ml_models/lstm/lstm_model.h5')